In [46]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
#Createing the connection to the database part 1
mongo = MongoClient(port=27017)
db = mongo['hockey_model']
db.list_collection_names()

['2022-2023', '2023-2023']

In [14]:
#creating connections part 2: putting it into a dataframe
data_2022_2023 = db['2022-2023']
answer = list(data_2022_2023.find({}))
df_2022 = pd.json_normalize(answer)

In [15]:
df_2022.head()

,_id,,playerId,season,name,team,position,situation,games_played,icetime,...,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,Line_or_Pair
0,659df24b1824a1bb36f2b0aa,0,8471817,2022,Ryan Reaves,MIN,R,all,73,40825,...,674.73,491,489.13,4.67,6.20,6.20,27.06,26.92,26.43,3rd line
1,659df24b1824a1bb36f2b0ab,1,8480950,2022,Ilya Lyubushkin,BUF,D,all,68,61236,...,1104.67,863,864.75,10.65,15.01,14.72,62.63,62.82,60.81,3rd pair
2,659df24b1824a1bb36f2b0ac,2,8475625,2022,Matt Irwin,WSH,D,all,61,47720,...,791.60,562,565.64,5.69,5.62,5.62,34.16,34.30,33.73,3rd pair
3,659df24b1824a1bb36f2b0ad,3,8480860,2022,Kevin Bahl,NJD,D,all,42,35331,...,508.74,378,379.33,3.76,7.03,7.03,21.02,21.10,20.84,3rd pair
4,659df24b1824a1bb36f2b0ae,4,8477952,2022,Robby Fabbri,DET,C,all,28,26877,...,425.03,316,313.93,3.19,2.47,2.47,20.77,20.86,20.65,2nd line


In [16]:
df_2022.drop(['_id','playerId','season','name','team','situation'], axis='columns',inplace=True)
df_2022.head()

,,position,games_played,icetime,minutes/game,onIce_xGoalsPercentage,onIce_corsiPercentage,onIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,...,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,Line_or_Pair
0,0,R,73,40825,9.32,0.45,0.45,0.46,814,49.47,...,674.73,491,489.13,4.67,6.20,6.20,27.06,26.92,26.43,3rd line
1,1,D,68,61236,15.01,0.39,0.42,0.41,329,37.10,...,1104.67,863,864.75,10.65,15.01,14.72,62.63,62.82,60.81,3rd pair
2,2,D,61,47720,13.04,0.44,0.45,0.46,338,69.79,...,791.60,562,565.64,5.69,5.62,5.62,34.16,34.30,33.73,3rd pair
3,3,D,42,35331,14.02,0.56,0.53,0.52,242,32.17,...,508.74,378,379.33,3.76,7.03,7.03,21.02,21.10,20.84,3rd pair
4,4,C,28,26877,16.00,0.52,0.46,0.46,139,39.69,...,425.03,316,313.93,3.19,2.47,2.47,20.77,20.86,20.65,2nd line


In [ ]:
df_2022.to_csv('model_2022-2023.csv')

# Preparing the Data

In [20]:
# Apply get_dummies on position and line/pair columns
dummies = pd.get_dummies(df_2022[['position','Line_or_Pair']])
# Drop the columns that get_dummies was used on
df_2022_col_drop = df_2022.drop(['position','Line_or_Pair'], axis=1)

# Concatenate the dummies df with the original df that had the columns dropped
df_2022_prepped = pd.concat([df_2022_col_drop, dummies], axis=1)

df_2022_prepped.head()

,,games_played,icetime,minutes/game,onIce_xGoalsPercentage,onIce_corsiPercentage,onIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,I_F_xGoals,...,position_C,position_D,position_L,position_R,Line_or_Pair_1st line,Line_or_Pair_1st pair,Line_or_Pair_2nd line,Line_or_Pair_2nd pair,Line_or_Pair_3rd line,Line_or_Pair_3rd pair
0,0,73,40825,9.32,0.45,0.45,0.46,814,49.47,6.70,...,0,0,0,1,0,0,0,0,1,0
1,1,68,61236,15.01,0.39,0.42,0.41,329,37.10,1.36,...,0,1,0,0,0,0,0,0,0,1
2,2,61,47720,13.04,0.44,0.45,0.46,338,69.79,2.44,...,0,1,0,0,0,0,0,0,0,1
3,3,42,35331,14.02,0.56,0.53,0.52,242,32.17,1.21,...,0,1,0,0,0,0,0,0,0,1
4,4,28,26877,16.00,0.52,0.46,0.46,139,39.69,6.81,...,1,0,0,0,0,0,1,0,0,0


In [21]:
#Splitting the data
X = df_2022_prepped.copy()
X.drop('I_F_points', axis=1, inplace=True)
y = df_2022_prepped['I_F_points'].ravel()

In [22]:
#further splitting into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12)

In [27]:
X_train.dtypes

                            int64
games_played                int64
icetime                     int64
minutes/game              float64
onIce_xGoalsPercentage    float64
                           ...   
Line_or_Pair_1st pair       uint8
Line_or_Pair_2nd line       uint8
Line_or_Pair_2nd pair       uint8
Line_or_Pair_3rd line       uint8
Line_or_Pair_3rd pair       uint8
Length: 143, dtype: object

In [28]:
#creating the scaler variable
scaler = StandardScaler()

In [32]:
X_scaler = scaler.fit(X_train)

In [33]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Linear Regression Model

In [39]:
linear_model = LinearRegression()

In [40]:
# Fit the data into the model
linear_model.fit(X_train_scaled, y_train)

LinearRegression()

In [41]:
# Display the model's best fit line formula
print(f"Model's formula: y = {linear_model.intercept_} + {linear_model.coef_[0]}X")

Model's formula: y = 28.525862068965516 + 2.02565150986641e-15X


In [42]:
#Make predictions with the model using X_test_scaled
predicted_y_values = linear_model.predict(X_test_scaled)

In [47]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = linear_model.score(X_test, y_test, sample_weight=None)
r2 = r2_score(y_test, predicted_y_values)
mse = mean_squared_error(y_test, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y_test)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is -122.01790216373101.
The r2 is 1.0.
The mean squared error is 2.6870830760570247e-26.
The root mean squared error is 1.6392324655328862e-13.
The standard deviation is 24.933234793302383.


c:\Users\evan-\anaconda3\envs\PythonData\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


# Random Forest

In [48]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=1000, random_state=12)

In [49]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [50]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [59]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = rf_model.score(X_test, y_test, sample_weight=None)
r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
std = np.std(y_test)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.005154639175257732.
The r2 is 0.8949033229817341.
The mean squared error is 65.33505154639175.
The root mean squared error is 8.083010054824364.
The standard deviation is 24.933234793302383.


c:\Users\evan-\anaconda3\envs\PythonData\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
